# Tesing BasicCatalogFilter 

The BasicCatalogFilter makes it possible to accept or reject alerts if there is a spatial coincidence with some sources of a given astronomical catalog. In this notebook, we illustrate how to use the BasicCatalogFilter to reject alerts nearby AGNs catalogued in the Million Quasar Catalog ([MQC](http://quasars.org/milliquas.htm))


For this example, we define a channel named HU_NOAGN by providing the channel configuration parameters. The way our example channel HU_NOAGN interacts with the BasicCatalogFilter is defined in a specific section of the channel config. In particular, you have to provide: 

* name of the catalog to be used for cross matching.
* name of the columns of the catalog containing Ra and Dec (e.g. 'RAJ2000', 'ra', ecc.)
* search radius (in arcsec)
* how the position of the alert has to be estimated, e.g. the all-epoch average or the current position of the alert. 
* search method for the catalog queries. This depends on the chosen indexing strategy for the catalog database.
* what to do when a match has been found within the search radius, i.e. either reject or accept the alert.

Here is the configuration of the BasicCatalogFilter we will demonstrate here:


```json
"t0Filter": {
    "id": "BasicCatalogFilter",
    "runConfig": {
        "catName": "milliquas",
        "catRAKey": "ra",
        "catDecKey": "dec",
        "searchMethod": "healpix",
        "rsArcsec": 3,
        "rejectOnMatch": true,
        "alertPosType": "av"
    }
```

The `rejectOnMatch` flag makes it possible to use catalog matching both to veto or to accept transients. That is, if you have a catalog of objects wich you are not (scientifically) interested in, you'd want to set the rejectOnMatch to True. In this case, a transitent will be accepted if no counterpart in the Boring Object Catalog is found. On the other hand, set rejectOnMatch to False if the catalog you're matching contains positoins of special interest to you. In this case, the alerts will be accepted if they match with the Really Cool Catalog.


To summarize the above settings: we will reject alerts if there is **at least one MQC** source within **3** arcsec from the all-epoch **average** position of the alert. The catalog ra/dec columns names are rather plain, and the queries routines will make use of the HEALPix index of the catalog.

------
**NOTE:**
The external catalogs to be examined have to be parsed into a database. The pacakage [extcats](https://github.com/MatteoGiomi/extcats) provides tools for setting up such a database and for querying it for positional matches. To run this example, git-clone the extcat package and run the setup script (the MQC is included in the test data).

In [1]:
# here some temp fixes
%env PYTHONPATH=../src/
import logging
logging.basicConfig(level=logging.INFO)

env: PYTHONPATH=../src/


## loading the channel:

In [4]:
from ampel.pipeline.t0.AlertProcessor import AlertProcessor
ap=AlertProcessor(load_channels=False)
ap.load_channel("HU_NOAGN")

INFO:Ampel-12:19:24.671482:Setting up new AlertProcessor instance
INFO:Ampel-12:19:24.671482:AlertProcessor initial setup completed
INFO:Ampel-12:19:24.671482:Loading filter: BasicCatalogFilter
INFO:Ampel-12:19:24.671482:   Full class path: ampel.pipeline.t0.filters.BasicCatalogFilter
INFO:Ampel-12:19:24.671482:using mongo client at localhost:27017
INFO:Ampel-12:19:24.671482:connected to collection srcs of database milliquas.
INFO:Ampel-12:19:24.671482:found 1998464 documents in source collection srcs.
INFO:Ampel-12:19:24.671482:set HEALPIX partition of order 16 with key hpxid_16. Nested: True, Inexed: True, Resolution ["]: 3.221
INFO:Ampel-12:19:24.671482:set 2dsphere key pos with format geoJSON. Inexed: True
INFO:Ampel-12:19:24.671482:source collection has the following indexes: _i, hpxid_16, pos_2dsphe
INFO:Ampel-12:19:24.671482:Initialized BasicCatalogFilter. Alerts with matches in milliquas (rs = 5.00 arcsec) will be accepted
INFO:Ampel-12:19:24.671482:   Version: 0.1
INFO:Ampel-1

### explanation of the log (need Valery's insight, could be wrong, misleading, or both):

As we are interested in testing one specific channel, the Alert Processor is instanciated with the load_channels = False. This makes it possible to manually load just our HU_NOAGN channel.

The BasicCatalogfilter object is loaded, and it sucessfully connects to the MQC database. Some statistic on this database is printed; the database indexes are examined.

## Run it!

now we apply the filter to some tests alerts and measure the average execution time.

In [5]:
import time, glob
test_alert_dir  = '/home/matteo/Desktop/20180209'

start = time.time()
ap.run_iterable_paths(base_dir = test_alert_dir)
end = time.time()

tspan = end - start
nalerts = len(glob.glob(test_alert_dir+'/*.avro'))
tav = tspan / float(nalerts)
print ("took %.2e sec to process %d alerts. On average %.3e sec / alert"%
       (tspan, nalerts, tav))

INFO:Ampel-12:19:24.671482:Returning iterable for file paths in folder: /home/matteo/Desktop/20180209
INFO:Ampel-12:19:24.671482:Executing run method
INFO:Ampel-12:19:24.671482:#######     Processing alerts     #######
INFO:Ampel-12:19:24.671482:Configuring ZIAlertIngester with channels ['HU_NOAGN']
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aaaxlwu
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aaaxmtv
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aabehey
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aaaxlhk
INFO:Ampel-12:19:24.671482

INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aabcffg
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aaaypvj
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aaazgub
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aaayeyw
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aabbewg
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NO

INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aaaxlmb
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aaawjbj
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aaazguc
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aabcfff
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aaaypvi
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NO

INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aabasuf
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aabdkbq
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aabcdot
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aabdsli
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aabcdic
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NO

INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aabdslk
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aabduol
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aaayrvg
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aabdjpa
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aaawoto
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NO

...

INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF17aaajozb
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF17aaajout
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF17aaapmtj
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF17aaajpbg
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF17aaajpbi
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NO

INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF17aaapukn
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF17aaajoru
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aabcyyo
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF17aaapmrg
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF17aaajpca
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NO

INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF18aabedfo
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF17aabtyoh
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF17aaajpbx
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF17aaapmtg
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Proces

INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF17aaapuki
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF17aaapukg
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF17aaajouz
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Processing alert: ZTF17aaajouf
INFO:Ampel-12:19:24.671482: -> Channel 'HU_NOAGN': alert was rejected
INFO:Ampel-12:19:24.671482:Disposing rejected candidates not implemented yet
INFO:Ampel-12:19:24.671482:Proces

took 5.41e+01 sec to process 4487 alerts. On average 1.205e-02 sec / alert
